In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [4]:
df = pd.read_csv('/content/fake_news_dataset.csv')
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [5]:
df.drop(["date", "source", "category"], axis=1, inplace=True)

In [6]:
df.head()

,title,text,author,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,Paula George,real
1,To offer down resource great point.,probably guess western behind likely next inve...,Joseph Hill,fake
2,Himself church myself carry.,them identify forward present success risk sev...,Julia Robinson,fake
3,You unit its should.,phone which item yard Republican safe where po...,Mr. David Foster DDS,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,Austin Walker,fake


In [7]:
df.shape

(3657, 4)

In [8]:
df["id"] = df.index

In [9]:
df.head()

,title,text,author,label,id
0,Foreign Democrat final.,more tax development both store agreement lawy...,Paula George,real,0
1,To offer down resource great point.,probably guess western behind likely next inve...,Joseph Hill,fake,1
2,Himself church myself carry.,them identify forward present success risk sev...,Julia Robinson,fake,2
3,You unit its should.,phone which item yard Republican safe where po...,Mr. David Foster DDS,fake,3
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,Austin Walker,fake,4


In [10]:
df.isnull().sum()

,0
title,0
text,0
author,209
label,1
id,0


In [11]:
df = df.fillna('')

In [12]:
df['content'] = df['title']+' '+df['author']

In [13]:
df['content']

,content
0,Foreign Democrat final. Paula George
1,To offer down resource great point. Joseph Hill
2,Himself church myself carry. Julia Robinson
3,You unit its should. Mr. David Foster DDS
4,Billion believe employee summer how. Austin Wa...
...,...
3652,Account admit how civil fish arrive safe. Scot...
3653,Good apply watch huge official what. Brian Cab...
3654,High adult suffer down experience. Denise Duncan
3655,Learn sea blood sell note. Madeline Ruiz


In [14]:
X = df.drop('label', axis=1)
Y = df['label']

In [15]:
ps = PorterStemmer()

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content) #Joins words back into a single cleaned string
    return stemmed_content

In [18]:
df['content'] = df['content'].apply(stemming)

In [19]:
print(df['content'])

0                      foreign democrat final paula georg
1                   offer resourc great point joseph hill
2                             church carri julia robinson
3                                 unit mr david foster dd
4             billion believ employe summer austin walker
                              ...                        
3652    account admit civil fish arriv safe scott mcclure
3653           good appli watch huge offici brian cabrera
3654                high adult suffer experi denis duncan
3655               learn sea blood sell note madelin ruiz
3656                       polic world final point across
Name: content, Length: 3657, dtype: object


In [38]:
X = df['content'].values
Y = df['label'].values

In [52]:
Vec = TfidfVectorizer()
X = Vec.fit_transform(df['content'].values)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24926 stored elements and shape (3657, 2093)>
  Coords	Values
  (0, 675)	0.42105636244981465
  (0, 487)	0.40713536114821725
  (0, 647)	0.38864899400415076
  (0, 1431)	0.5108795645032354
  (0, 725)	0.49488015455844164
  (1, 1372)	0.4044325857861431
  (1, 1574)	0.39867655544363995
  (1, 758)	0.3959760736534473
  (1, 1471)	0.39088489933467996
  (1, 982)	0.3817458928319161
  (1, 838)	0.4713526339864967
  (2, 359)	0.4617185490439717
  (2, 304)	0.45869323198384276
  (2, 992)	0.5858636647810592
  (2, 1609)	0.482887426519114
  (3, 1955)	0.4850091146106192
  (3, 1311)	0.3448867496126958
  (3, 467)	0.3700062826443203
  (3, 680)	0.5231451024663881
  (3, 472)	0.4850091146106192
  (4, 190)	0.3918051730628542
  (4, 163)	0.40466558898503646
  (4, 577)	0.41635006063422303
  (4, 1849)	0.3862288606099006
  (4, 114)	0.4362270106249323
  :	:
  (3653, 267)	0.4487445252010913
  (3653, 235)	0.3419847677561803
  (3653, 1374)	0.3419847677561803
  (3

In [42]:
X_train, X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.2 , random_state=2)


In [43]:
model = LogisticRegression()

In [44]:
model.fit(X_train, Y_train)



LogisticRegression()

In [45]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [46]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8034188034188035


In [47]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [48]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.4890710382513661


In [53]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['real']
The news is Fake
